In [ ]:
#| default_exp mingpt
%load_ext autoreload
%autoreload 2

In [ ]:
do_download:bool=False
if do_download:
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# | export
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd

In [ ]:
is_dylan: bool = False
if is_dylan:
    df = pd.read_csv("./bob_dylan_lyrics.csv")
    text = "\n\n".join(df.lyrics.values.tolist())
else:
    with open("./input.txt", "r", encoding="utf-8") as f:
        text = f.read()

    len(text), text[:100]
text

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}


def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]


def decode(l: list[int]) -> str:
    return "".join([itos[i] for i in l])


decode(encode("hello world"))
encode("hello world")

In [ ]:
# torch.tensor()
data = torch.tensor(encode(text), dtype=torch.long)
data[:100]
n = int(0.9 * len(text))
train = data[:n]
val = data[n:]

In [ ]:
val

In [ ]:
block_size = 8

In [ ]:
x = train[:block_size]
y = train[1 : block_size + 1]
x, y
for t in range(block_size):
    ctx = x[: t + 1]
    tgt = y[t]
    print(t, ctx, tgt)

In [ ]:
torch.manual_seed(1337)
batch_size = 3
block_size = 8


def get_batch(split: str):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # ix are random starting points of each batch
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

for b in range(batch_size):
    for t in range(block_size):
        ctx = xb[b, 0 : t + 1]
        tgt = yb[b, t]
        print(ctx, "->", tgt)

In [ ]:
xb, yb

In [ ]:
torch.manual_seed(42)


class BigramModel(nn.Module):
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_emb = torch.nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_emb(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            print(targets.shape)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # print(idx.shape, idx[:,-1].unsqueeze(1).shape)
            logits, _ = self(idx)

            logits = logits[:, -1, :]
            # print(logits.shape)
            probs = F.softmax(logits, dim=-1)
            # print(probs.shape, idx.shape)
            idx_next = torch.multinomial(probs, num_samples=1)
            # print(idx_next.shape)
            # idx_next = probs.argmax(-1).unsqueeze(1)
            # print(idx_next.shape)
            idx = torch.cat((idx, idx_next), dim=1)
            # print(">>",idx.shape)
        return idx

    def generate_bigram(self, idx, max_new_tokens):
        ...


m = BigramModel(vocab_size)
logits, loss = m(xb, yb)
print(f"loss {loss:.4f}")

#

o = m.generate(xb, max_new_tokens=2)
o.shape

In [ ]:
idx = torch.ones(1, 1, dtype=torch.long)
decode(m.generate(idx, max_new_tokens=100)[0].tolist())

In [ ]:
torch.tensor([[0.2, 0.2, 0.6], [0.5, 0.2, 0.3]]).argmax(-1)

In [ ]:
B, T, C = l.shape
l.view(B * T, -1).shape

In [ ]:
emb.weight

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export("mingpt.ipynb")